# Filtrado de señales

## Paquetes y gráficas

In [ ]:
#   Importar cosas que importan
import numpy as np
from sympy import symbols, latex, sympify, Number
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, lfilter, freqz, TransferFunction
from scipy.io.wavfile import read, write
from scipy import fftpack
from IPython.display import display, Math, Audio, Latex

In [ ]:
#   Funciones para graficar
def graph(graph_type, *data, width = 12, height = 7):
    plt.figure(figsize=(width, height))
    if graph_type == 'waveform':
        plt.plot(data[0], color='xkcd:lightish blue', label='Original signal')
        if len(data) > 1: plt.plot(data[1], color='xkcd:watermelon', label='Filtered signal')
        plt.xlabel('Sample Index')
        plt.ylabel('Amplitude')
        plt.title('Waveform')
    elif graph_type == 'fft':
        plt.plot(data[0], data[1], color='xkcd:lightish blue', label='Original signal')
        if len(data) > 2: plt.plot(data[2], data[3], color='xkcd:watermelon', label='Filtered signal')
        plt.xlabel('Frequency')
        # plt.xscale('log')
        plt.ylabel('Amplitude')
        # plt.yscale('log')
        plt.title('Fourier Spectrum')
    # plt.gca().spines['top'].set_position(('data',0))
    # plt.gca().spines['right'].set_position(('data',0))
    plt.grid(ls='--', zorder=0)
    plt.legend(loc='upper right')
    # plt.gca().set_xlim([-100,6000])
    plt.show()

## Señales de entrada

In [ ]:
#   Señal de ejemplo
##  Parámetros
T = 5           # Sample period (s)
fs = 300.0      # Sample freq (Hz)
n = int(T * fs) # Total number of samples
t = np.linspace(0,T,n, endpoint=True)

sig = np.sin(1.2*2*np.pi*t)
noise = 1.5*np.cos(9*2*np.pi*t) + 0.5*np.sin(12.0*2*np.pi*t)    # Ruido
data = sig + noise

In [ ]:
#   Audio 1
fs, data = read('1Hz_44000Hz_-3dBFS_30s.wav')               # Sample freq (Hz), data
                                           # Currently "voice.wav", "drums.wav", "noise.wav"
if len(data.shape) > 1: data = data[:,0]   # Select only one channel

##  Parámetros
n = data.shape[0]               # Total samples
T = n/fs                        # Sample period (s)
t = np.linspace(0,T,n, endpoint=True)

Audio(data, rate=fs)

### Visualización

In [ ]:
#   Waveform
graph('waveform', data)

In [ ]:
#   Fourier Spectrum
sig_noise_fft = fftpack.fft(data)
sig_noise_amp = 2 / t.size * np.abs(sig_noise_fft)
sig_noise_freq = np.abs(fftpack.fftfreq(t.size, T/n))

graph('fft', sig_noise_freq, sig_noise_amp)

## Filtro de Butterworth

### Cálculo y demás

In [ ]:
def butter_lowpass_filter(data, cutoff, nyq, order):
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y, b, a

In [ ]:
#   Parámetros
cutoff = 20000  # Desired cutoff frequency of the filter, Hz
                # Use the Fourier Spectrum to define desired cutoff frequency
                # Example uses 2Hz, voice around 3KHz
nyq = 0.5 * fs  # Nyquist Frequency
order = 3

#   Filter the data
filtered_data, b, a = butter_lowpass_filter(data, cutoff, nyq, order)

### Visualización y exportación

In [ ]:
#   Plot both the original and filtered signals, filtered audio
graph('waveform', data, filtered_data)

In [ ]:
#   Fourier spectrum of filtered audio
sig_filter_fft = fftpack.fft(filtered_data)
sig_filter_amp = 2 / t.size * np.abs(sig_filter_fft)
sig_filter_freq = np.abs(fftpack.fftfreq(t.size, T/n))

graph('fft', sig_noise_freq, sig_noise_amp, sig_filter_freq, sig_filter_amp)

In [ ]:
#   Output filtered audio
Audio(filtered_data, rate=fs)
write('output.wav', fs, filtered_data.astype(np.int16))

### Diagramas de Bode

In [ ]:
colors = ['xkcd:violet', 'xkcd:blue', 'xkcd:neon blue', 'xkcd:green', 'xkcd:yellowish green', 'xkcd:dark yellow', 'xkcd:orange', 'xkcd:red', 'xkcd:pink']  # Add more xkcd colors if needed

plt.figure(figsize=(12,7))
for i in range(1,11):
    _, bb, aa = butter_lowpass_filter(data, cutoff, nyq, i)
    w, h = freqz(bb, aa, fs = fs)

    plt.plot(w, np.abs(h), color=colors[i%len(colors)-1], label=f'Orden {i}')
    plt.plot(cutoff, 0.5*np.sqrt(2), 'ko')
    plt.axvline(cutoff, color='k')
    plt.xlim(0, 0.5*fs)
    
plt.title("Diagrama de Bode - Filtro de paso bajo")
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Magnitud [dB]')
plt.gca().spines['top'].set_position(('data',0))
plt.gca().spines['right'].set_position(('data',0))
plt.grid(ls='--', zorder=0)
plt.legend(loc='upper right')
plt.savefig("bode.png", dpi = 300)
plt.show()

### Función de transferencia

In [ ]:
tf = TransferFunction(b, a)

#   Pretty print the transfer function in LaTeX
def pretty_print_tf_latex(tf):
    s = symbols('s')
    num = sum(coeff * s**power for power, coeff in enumerate(tf.num[::-1]))
    den = sum(coeff * s**power for power, coeff in enumerate(tf.den[::-1]))
    latex_str = (sympify(num / den))
    formatted_str = latex_str.xreplace({n : round(n, 3) for n in latex_str.atoms(Number)})
    return latex(formatted_str)

# Use the function
display(Math(rf'H(s) = {pretty_print_tf_latex(tf)}'))